In [2]:
# Chris comments - imports should be organized by standard library first,
# then third party, then custom code
import json
import math

import folium
from folium import FeatureGroup, LayerControl
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Polygon, Point, MultiPolygon

## just an example of importing your custom code, which would go here
# from .minard import functions 

In [3]:
# Chris: until we have a reliable pipeline to accomplish our MVP, 
# I suggest we don't include the 'cities' or 'temperatures' info.
# The more tables we use, the more info the user needs to provide,
# and the more complicated our data processing becomes
# I also rename 'troops' to 'troops_df' and 'gdf_troops' to 'troops_gdf'. 
# This is just preference and a practice I've seen elsewhere.

# cities = pd.read_csv("data/cities.csv", sep=" ", names=["lon", "lat", "city"])
# temperatures = pd.read_csv("data/temperature.txt", sep=" ", names=["lon", "temp", "days", "day"])
troops_df = pd.read_csv("data/troops.txt", sep=" ", 
                        names=["lon", "lat", "survivors", 
                               "direction", "division"])

In [4]:
# Chris: troops_df is already a pd.DataFrame, so this isn't necessary:
# df_troops = pd.DataFrame(troops)

troops_gdf = gpd.GeoDataFrame(
    troops_df, 
    crs={'init': 'epsg:4326'}, 
    geometry=[Point(xy) for xy in zip(troops_df.lon, troops_df.lat)]
)

In [6]:
## Chris: this processing can be done inside the `draw_polygons` function
# coords = []
# for i, row in troops_gdf.iterrows():
#     coords.append([row.geometry.y, row.geometry.x])

# min_lon, min_lat = [coord * .98 for coord in min(coords)] 
# max_lon, max_lat = [coord * 1.02 for coord in max(coords)] 

# center_lat = (min_lat + max_lat) / 2
# center_lon = (min_lon + max_lon) / 2

**Chris:** In the `draw_polygons` function below I change the input `gdf_troops` to `input_gdf` because we want our functions to generalize beyond just this specific example. Think about what other arguments we will need to make the function more generalizable. For example, the column names `'survivors'`, `'direction'` and labels `'attack'`/`'retreat'` are all specific to  the minards dataset. What is a more general name for `'survivors'` in our use case? How can we change this function so the user can tell us what this column is called in their data? Likewise for `'direction'` - what other info might be used in the  same way in other contexts? If this were the CTA, this column might be `'route_name'` and include values like `'red', 'brown', 'purple'`.

I also changed the transparency just to help me visually see what is plotted.

Also, instead of iterating through rows or indices, I suggest some **vectorized** computation, which would be more efficient with larger datasets. For example, when we compare the current point to the next point, we could do that with vectorization like so:

In [7]:
# assign columns to represent the next point for each row
troops_gdf['lon1'] = troops_gdf['geometry'].x
troops_gdf['lat1'] = troops_gdf['geometry'].y
troops_gdf[['lon2', 'lat2']] = troops_gdf[['lon1', 'lat1']].shift(-1)

troops_gdf['lon_diff'] = troops_gdf['lon2'] - troops_gdf['lon1']
troops_gdf['lat_diff'] = troops_gdf['lat2'] - troops_gdf['lat1']
troops_gdf['slope'] = troops_gdf['lat_diff'] / troops_gdf['lon_diff']
troops_gdf.head()

# Following this, we might assign columns to our x1, x2, y1, y2, 
# then inverse slope, buffer, color, etc. until we have a column that 
# is the polygon we will draw on the map.

,lon,lat,survivors,direction,division,geometry,lon1,lat1,lon2,lat2,lon_diff,lat_diff,slope
0,24.0,54.9,340000,A,1,POINT (24 54.9),24.0,54.9,24.5,55.0,0.5,0.1,0.2
1,24.5,55.0,340000,A,1,POINT (24.5 55),24.5,55.0,25.5,54.5,1.0,-0.5,-0.5
2,25.5,54.5,340000,A,1,POINT (25.5 54.5),25.5,54.5,26.0,54.7,0.5,0.2,0.4
3,26.0,54.7,320000,A,1,POINT (26 54.7),26.0,54.7,27.0,54.8,1.0,0.1,0.1
4,27.0,54.8,300000,A,1,POINT (27 54.8),27.0,54.8,28.0,54.9,1.0,0.1,0.1


In [8]:
def draw_polygons(mapobj, input_gdf):
    # Chris: add documentation string here:
    """" """
    
    ## Chris: since we aren't using the 'row' that we get here, this might
    ## not be the best iteration technique:
    # for i, row in input_gdf.iterrows():
    
    ## instead, we might use this. Then we don't need the if/else blocks to
    ## validate our index
    for i in range(len(input_gdf) - 1):
        ## Chris: 'coords' is a varialbe that is outside of the function scope,
        ## meaning it is not passed into the function as an argument, and it is
        ## not defines inside the function. This should be avoided:
        # x1, y1 = coords[i]
        # x2, y2 = coords[i+1]

        ## Chris: this would work instead here
        x1, y1 = input_gdf.geometry.x[i], input_gdf.geometry.y[i]
        x2, y2 = input_gdf.geometry.x[i+1], input_gdf.geometry.y[i+1]
        
        if y2 - y1 == 0: # Chris: had to add this condition
            slope_inv = 1e10
        elif x2 - x1 != 0:
            #slope of line
            slope = (y2 - y1) / (x2 - x1)

            #negative inverse slope (perpindicular angle)
            slope_inv = (-1 / slope)
        else:
            slope = None
            slope_inv = 0

        #cosine, sine to find x, y coordinates of polygon corners
        c = 1 / math.sqrt(1 + (slope_inv**2))
        s = slope_inv / math.sqrt(1 + (slope_inv**2))

        #find width of polygon (magnitude of survivors)
        #abstract max survivors
        #move outside of function and make new column
        buffer = (input_gdf['survivors'][i] / 34000) *.03

        x1u, y1u = (x1 + (buffer*c)), (y1 + (buffer*s))
        x1d, y1d = (x1 - (buffer*c)), (y1 - (buffer*s))
        x2u, y2u = (x2 + (buffer*c)), (y2 + (buffer*s))
        x2d, y2d = (x2 - (buffer*c)), (y2 - (buffer*s))

        lon_point_list = [(x1 + (buffer*c)), 
                          (x1 - (buffer*c)), 
                          (x2 - (buffer*c)), 
                          (x2 + (buffer*c))]
        lat_point_list = [(y1 + (buffer*s)), 
                          (y1 - (buffer*s)), 
                          (y2 - (buffer*s)), 
                          (y2 + (buffer*s))]

        fg_attack = folium.FeatureGroup(name='attack').add_to(mapobj)
        fg_retreat = folium.FeatureGroup(name='retreat').add_to(mapobj)

        geometry = list(zip(lat_point_list, lon_point_list))


        if troops_gdf['direction'][i] == 'A':
            folium.Polygon(geometry,
                       color = None,
                       fill_color = '#F0B683', 
                       fill_opacity = 0.8).add_to(fg_attack)
        else:
            folium.Polygon(geometry,
                       color = None,
                       fill_color = 'black', 
                       fill_opacity = 0.8).add_to(fg_retreat)

        mapobj.keep_in_front(fg_attack)
    
    return mapobj

**Chris**: it looks like we still have some issues in our plot. This is because of a few things:

1. We are using euclidian computations on a non-flat globe. 
2. We are using the iverse slope for each segment to define the left and right edges. We might want to use the inverse slopre of the **average slope of the two segments**.
3. We assumed that the data are already sorted (which appears to be true here, but is something to think about going forward)
4. We assumed that each row is connected to the next row regardless of group/direction

In [9]:
m = folium.Map(location=[54,31],
               tiles='OpenStreetMap',
               zoom_start=6,
               control_scale=True)

m = draw_polygons(m, troops_gdf)
m

![](https://upload.wikimedia.org/wikipedia/commons/2/29/Minard.png)